In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

import re, string
import nltk

from sklearn import preprocessing
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split

from transformers import AutoTokenizer, TFAutoModel


from tensorflow import keras


from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
df= pd.read_csv('/kaggle/input/twitter-data/processed_twitter_data.csv')
print(df.head())

                                      processed_text  sentiment
0  USER URL aww that bummer you shoulda got david...          0
1  is upset that he can update his facebook by te...          0
2  USER dived many time for the ball managed to s...          0
3      my whole body feel itchy and like it on fire           0
4  USER no it not behaving at all mad why am here...          0


In [3]:
tokenizer = AutoTokenizer.from_pretrained('vinai/bertweet-base', use_fast=True)

config.json:   0%|          | 0.00/558 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/843k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.91M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
X_train, X_test, y_train, y_test = train_test_split(df['processed_text'], df['sentiment'], test_size=0.2, random_state=0)
print(f'Data Split into Training and Test sets done.')

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
print(f'Training set split into Training and Validation sets done.')

Data Split into Training and Test sets done.
Training set split into Training and Validation sets done.


In [5]:
MAX_LEN=100

In [6]:
def tokenize(data,max_len=MAX_LEN) :
    input_ids = []
    attention_masks = []
    for i in range(len(data)):
        encoded = tokenizer.encode_plus(
            data.iloc[i],
            add_special_tokens=True,
            max_length=MAX_LEN,
            padding='max_length',
            return_attention_mask=True
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    return np.array(input_ids),np.array(attention_masks)

In [7]:
train_input_ids, train_attention_masks = tokenize(X_train, MAX_LEN)
val_input_ids, val_attention_masks = tokenize(X_valid, MAX_LEN)
test_input_ids, test_attention_masks = tokenize(X_test, MAX_LEN)

In [8]:
berttweet_model = TFAutoModel.from_pretrained('vinai/bertweet-base')

tf_model.h5:   0%|          | 0.00/740M [00:00<?, ?B/s]

Some layers from the model checkpoint at vinai/bertweet-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at vinai/bertweet-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


In [9]:
def create_model(bert_model, max_len=MAX_LEN):
    
    ##params###
    opt = tf.keras.optimizers.Adam(learning_rate=1e-5)
    loss = tf.keras.losses.BinaryCrossentropy()
    accuracy = tf.keras.metrics.BinaryAccuracy()


    input_ids = tf.keras.Input(shape=(max_len,),dtype='int32')
    
    attention_masks = tf.keras.Input(shape=(max_len,),dtype='int32')
    
    embeddings = bert_model([input_ids,attention_masks])[0]
    
    output = tf.keras.layers.Dense(1, activation="sigmoid")(embeddings[:, 0, :])
    
    model = tf.keras.models.Model(inputs = [input_ids,attention_masks], outputs = output)
    
    model.compile(opt, loss=loss, metrics=accuracy)
    
    
    return model

In [10]:
model = create_model(berttweet_model, MAX_LEN)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 100)]                0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 100)]                0         []                            
                                                                                                  
 tf_roberta_model (TFRobert  TFBaseModelOutputWithPooli   1348999   ['input_1[0][0]',             
 aModel)                     ngAndCrossAttentions(last_   68         'input_2[0][0]']             
                             hidden_state=(None, 100, 7                                           
                             68),                                                             

In [11]:
bertweet_mod = model.fit([train_input_ids,train_attention_masks], y_train, validation_data=([val_input_ids,val_attention_masks], y_valid), epochs=3, batch_size=64)

Epoch 1/3


I0000 00:00:1727501455.292522     100 service.cc:145] XLA service 0x7fb20b0915b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1727501455.292572     100 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1727501455.464469     100 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


16000/16000 [==============================] - 11975s 744ms/step - loss: 0.3168 - binary_accuracy: 0.8625 - val_loss: 0.2972 - val_binary_accuracy: 0.8723
Epoch 2/3
16000/16000 [==============================] - 11900s 744ms/step - loss: 0.2791 - binary_accuracy: 0.8807 - val_loss: 0.3119 - val_binary_accuracy: 0.8682
Epoch 3/3
 4439/16000 [=======>......................] - ETA: 2:12:16 - loss: 0.2488 - binary_accuracy: 0.8955

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



16000/16000 [==============================] - 11887s 743ms/step - loss: 0.2500 - binary_accuracy: 0.8947 - val_loss: 0.3150 - val_binary_accuracy: 0.8733


In [12]:
import pickle

with open('Sentiment-BERT-tweet.pickle', 'wb') as file:
    pickle.dump(bertweet_mod, file)

/opt/conda/lib/python3.10/site-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


In [13]:
model.save('bertweet_sentiment_model.h5')

/opt/conda/lib/python3.10/site-packages/tf_keras/src/engine/training.py:3098: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native TF-Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [14]:
result_bertweet = model.predict([test_input_ids,test_attention_masks])

10000/10000 [==============================] - 1271s 126ms/step


In [15]:
y_pred_bertweet = (result_bertweet > 0.5).astype(int)

In [16]:
y_pred_bertweet = y_pred_bertweet.flatten()

In [17]:
conf_matrix = confusion_matrix(y_test, y_pred_bertweet)
print('Confusion Matrix:\n', conf_matrix)

Confusion Matrix:
 [[141610  18205]
 [ 22129 138056]]


In [18]:
print('\tClassification Report for BERTweet:\n\n', classification_report(y_test, y_pred_bertweet, target_names=['Negative', 'Positive']))

	Classification Report for BERTweet:

               precision    recall  f1-score   support

    Negative       0.86      0.89      0.88    159815
    Positive       0.88      0.86      0.87    160185

    accuracy                           0.87    320000
   macro avg       0.87      0.87      0.87    320000
weighted avg       0.87      0.87      0.87    320000



In [19]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

accuracy = accuracy_score(y_test, y_pred_bertweet)

precision = precision_score(y_test, y_pred_bertweet)
recall = recall_score(y_test, y_pred_bertweet)
f1 = f1_score(y_test, y_pred_bertweet)

print('BERTweet Model Metrics')
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

BERTweet Model Metrics
Accuracy: 0.8740
Precision: 0.8835
Recall: 0.8619
F1 Score: 0.8725
